# Flood Susceptibility Mapping with Random Forest
## Artvin Province, Turkiye - MYZ 305E GeoAI Applications
**Authors:** Mevlutcan Yildizli, Ugur Ince | **ITU Fall 2025**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import ndimage
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
print('Libraries imported successfully!')

In [ ]:
# Study Area Configuration
STUDY_AREA = {'name': 'Artvin Province', 'country': 'Turkiye', 'area_km2': 7436}
BBOX = (41.0, 40.7, 42.5, 41.7)
print(f'Study Area: {STUDY_AREA["name"]}, {STUDY_AREA["country"]}')

In [ ]:
# Simulate DEM for Artvin
np.random.seed(42)
shape = (500, 500)
x, y = np.meshgrid(np.linspace(0,1,shape[1]), np.linspace(0,1,shape[0]))
dem = 3900 * (1-y)**0.5 + np.random.normal(0,100,shape)
valley = 0.5 + 0.1*np.sin(y*np.pi*3)
dem -= np.exp(-(x-valley)**2/0.02)*500
dem = np.clip(dem, 0, 3900).astype(np.float32)

# Compute derivatives
kx = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])/(8*30)
ky = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])/(8*30)
slope = np.degrees(np.arctan(np.sqrt(ndimage.convolve(dem,kx)**2 + ndimage.convolve(dem,ky)**2)))
flow = np.max(dem)-ndimage.uniform_filter(dem,5)+1
twi = np.clip(np.log(flow*30/np.tan(np.radians(np.maximum(slope,0.1)))), -5, 25)

print(f'DEM: {dem.min():.0f}-{dem.max():.0f}m, Slope: {slope.max():.1f}deg')

In [ ]:
# Visualize terrain
fig, axes = plt.subplots(1,3,figsize=(15,4))
for ax,data,title,cmap in zip(axes,[dem,slope,twi],['DEM','Slope','TWI'],['terrain','YlOrRd','Blues']):
    im = ax.imshow(data, cmap=cmap)
    ax.set_title(title, fontweight='bold')
    plt.colorbar(im, ax=ax, shrink=0.8)
plt.tight_layout()
plt.savefig('terrain_analysis.png', dpi=150)
plt.show()

In [ ]:
# Generate training samples
n_samples = 847
flood_pts = [(int(shape[0]*np.random.uniform(0.1,0.9)), int(shape[1]*0.5+np.random.normal(0,shape[1]*0.1))) for _ in range(n_samples)]
non_flood_pts = [(int(shape[0]*np.random.uniform(0.1,0.9)), int(shape[1]*(0.15 if np.random.rand()>0.5 else 0.85)+np.random.normal(0,shape[1]*0.05))) for _ in range(n_samples)]

X = np.array([[dem[np.clip(p[0],0,499),np.clip(p[1],0,499)], slope[np.clip(p[0],0,499),np.clip(p[1],0,499)], twi[np.clip(p[0],0,499),np.clip(p[1],0,499)]] for p in flood_pts+non_flood_pts])
y = np.array([1]*n_samples + [0]*n_samples)
print(f'Samples: {len(y)} (Flood: {sum(y)}, Non-flood: {sum(y==0)})')

In [ ]:
# Train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_s, X_test_s = scaler.fit_transform(X_train), scaler.transform(X_test)

rf = RandomForestClassifier(n_estimators=300, max_depth=20, class_weight='balanced', random_state=42, n_jobs=-1)
cv_auc = cross_val_score(rf, X_train_s, y_train, cv=5, scoring='roc_auc')
print(f'CV AUC: {cv_auc.mean():.3f} +/- {cv_auc.std():.3f}')

rf.fit(X_train_s, y_train)
y_pred, y_prob = rf.predict(X_test_s), rf.predict_proba(X_test_s)[:,1]
print(f'Test AUC: {roc_auc_score(y_test, y_prob):.3f}, F1: {f1_score(y_test, y_pred):.3f}')

In [ ]:
# Evaluation plots
fig, axes = plt.subplots(1,3,figsize=(14,4))

# Confusion matrix
sns.heatmap(confusion_matrix(y_test,y_pred), annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title('Confusion Matrix', fontweight='bold')

# ROC curve
fpr, tpr, _ = roc_curve(y_test, y_prob)
axes[1].plot(fpr, tpr, 'b-', lw=2)
axes[1].plot([0,1],[0,1],'k--')
axes[1].set_title(f'ROC Curve (AUC={roc_auc_score(y_test,y_prob):.3f})', fontweight='bold')

# Feature importance
imp = pd.DataFrame({'Feature':['Elevation','Slope','TWI'], 'Importance':rf.feature_importances_}).sort_values('Importance')
axes[2].barh(imp['Feature'], imp['Importance']*100, color=plt.cm.RdYlGn_r(np.linspace(0.2,0.8,3)))
axes[2].set_title('Feature Importance', fontweight='bold')

plt.tight_layout()
plt.savefig('model_evaluation.png', dpi=150)
plt.show()

In [ ]:
# Generate susceptibility map
X_full = np.stack([dem.flatten(), slope.flatten(), twi.flatten()], axis=1)
susc = rf.predict_proba(scaler.transform(X_full))[:,1].reshape(shape)
classified = np.digitize(susc, np.percentile(susc, [20,40,60,80]))

fig, axes = plt.subplots(1,2,figsize=(14,5))
im1 = axes[0].imshow(susc, cmap='RdYlGn_r', vmin=0, vmax=1)
axes[0].set_title('Flood Susceptibility (Probability)', fontweight='bold')
plt.colorbar(im1, ax=axes[0], shrink=0.8)

from matplotlib.colors import ListedColormap
cmap = ListedColormap(['#1a9641','#a6d96a','#ffffbf','#fdae61','#d7191c'])
im2 = axes[1].imshow(classified, cmap=cmap)
axes[1].set_title('Flood Susceptibility (Classified)', fontweight='bold')
import matplotlib.patches as mpatches
patches = [mpatches.Patch(color=c, label=l) for c,l in zip(['#1a9641','#a6d96a','#ffffbf','#fdae61','#d7191c'], ['Very Low','Low','Moderate','High','Very High'])]
axes[1].legend(handles=patches, loc='lower right')

plt.tight_layout()
plt.savefig('flood_susceptibility_map.png', dpi=300)
plt.show()

high_risk = ((classified>=3).sum()/classified.size)*100
print(f'High/Very High Risk Areas: {high_risk:.1f}%')

In [ ]:
print('='*50)
print('SUMMARY')
print('='*50)
print(f'Study Area: Artvin Province, Turkiye (7436 km2)')
print(f'Model: Random Forest (300 trees, depth=20)')
print(f'AUC-ROC: {roc_auc_score(y_test, y_prob):.3f}')
print(f'F1-Score: {f1_score(y_test, y_pred):.3f}')
print(f'High Risk Area: {high_risk:.1f}%')
print('='*50)